In [19]:
from chemdataextractor.doc import Document, Heading, Paragraph, Sentence
import os
import json
import pandas as pd
import csv
import requests
from nltk import sent_tokenize
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
doi_index = []
text = []

In [3]:
pathacs = '/Users/juliachotoo/ScrapyArticles/scrapedjson/acs2/'

for filename in os.listdir(pathacs):
    with open(pathacs + filename, 'r'):
        resultacs = json.load(open(pathacs + filename, mode='r'))
        doi_index.append(resultacs['doi'])
        text.append(resultacs['text'])

pathspr = '/Users/juliachotoo/ScrapyArticles/scrapedjson/spr2/'

for filename in os.listdir(pathspr):
    with open(pathspr + filename, 'r'):
        resultspr = json.load(open(pathspr + filename, mode='r'))
        doi_index.append(resultspr['doi'])
        text.append(resultspr['text'])

pathlabels = '/Users/juliachotoo/ScrapyArticles/datalabels.csv'
labels = []

with open(pathlabels, 'r') as csvfile:
    resultlabels = csv.reader(csvfile, delimiter=' ', quotechar='|')
    for row in resultlabels:
        labels.append(', '.join(row))

labels[0] = labels[0].replace('\ufeff', '')

In [4]:
df = pd.DataFrame({'data': text, 'labels': labels})
df = df[~df['labels'].str.contains('2')]
#print(df.loc[0:9])

In [5]:
host = 'http://dirac.nist.gov/adsorption.nist.gov'
URL = host + '/srd205/api/gases.json'
gas_library = json.loads(requests.get(URL).content.decode('utf-8'))

In [21]:
#print(gas_library)

In [6]:
doc = Document(text[0])

In [7]:
#names = doc.records.serialize()
#print(names)
names = doc.cems


In [8]:
print(names)

[Span('CH4', 606, 609), Span('11282-11285CODEN', 10347, 10363), Span('CO2', 6530, 6533), Span('nitro', 8345, 8350), Span('2,6-DNT', 8467, 8474), Span('triphenylene-2,6,10-tricarboxylic acid (H3TTCA)(7)', 1541, 1591), Span('CH4', 8900, 8903), Span('1,4-dioxane', 1643, 1654), Span('TTCA', 2607, 2611), Span('TNT', 8689, 8692), Span('N2', 4203, 4205), Span('CO2', 5927, 5930), Span('CO2', 6200, 6203), Span('CO2', 6440, 6443), Span('CO2', 6518, 6521), Span('CH4', 6522, 6525), Span('CH4', 4949, 4952), Span('N2', 729, 731), Span('In(NO3)3·H2O', 1597, 1609), Span('2,4-DNT', 8694, 8701), Span('CO2', 297, 300), Span('CO2', 8985, 8988), Span('CH4', 5065, 5068), Span('CO2', 5098, 5101), Span('CO2', 1413, 1416), Span('N2', 5823, 5825), Span('2,4-DNT', 7841, 7848), Span('CO2', 652, 655), Span('N2', 5735, 5737), Span('CO2', 8891, 8894), Span('N2', 4958, 4960), Span('DMF', 3516, 3519), Span('N2', 6454, 6456), Span('N2', 6534, 6536), Span('H2O', 3524, 3527), Span('CO2', 474, 477), Span('nitro', 1473, 14

In [9]:
print(names[0:2])

[Span('CH4', 606, 609), Span('11282-11285CODEN', 10347, 10363)]


In [8]:
dir(names[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'end',
 'length',
 'start',
 'text']

In [8]:
print(names[0].__dict__)

{'text': 'DMF', 'start': 1352, 'end': 1355}


In [9]:
print(names[9].text)

N2


In [8]:
gas_names = []
for y in gas_library:
    for x in y['Synonyms']:
        gas_names.append(x)

In [9]:
cross_ref = []
for x in gas_names:
    for i in range(len(names)):
        if names[i].text.lower() == x.lower():
            #print(names[i].text)
            cross_ref.append(names[i])
print(cross_ref)

[Span('CO2', 4944, 4947), Span('CO2', 5056, 5059), Span('CO2', 5806, 5809), Span('CO2', 6518, 6521), Span('CO2', 5711, 5714), Span('CO2', 4592, 4595), Span('CO2', 6000, 6003), Span('CO2', 1413, 1416), Span('CO2', 5525, 5528), Span('CO2', 9018, 9021), Span('CO2', 4835, 4838), Span('CO2', 6530, 6533), Span('CO2', 6132, 6135), Span('CO2', 577, 580), Span('CO2', 652, 655), Span('CO2', 5448, 5451), Span('CO2', 4651, 4654), Span('CO2', 8891, 8894), Span('CO2', 6200, 6203), Span('CO2', 6440, 6443), Span('CO2', 5927, 5930), Span('CO2', 5731, 5734), Span('CO2', 474, 477), Span('CO2', 700, 703), Span('CO2', 6399, 6402), Span('CO2', 297, 300), Span('CO2', 8985, 8988), Span('CO2', 5098, 5101), Span('N,N-dimethylformamide', 1358, 1379), Span('H2', 4453, 4455), Span('H2', 4794, 4796), Span('hydrochloric acid', 1614, 1631), Span('CH4', 6445, 6448), Span('CH4', 606, 609), Span('CH4', 5815, 5818), Span('CH4', 5943, 5946), Span('CH4', 1433, 1436), Span('CH4', 5065, 5068), Span('CH4', 5715, 5718), Span('

In [10]:
sentences = sent_tokenize(text[0])


In [11]:
important_sent =[]
for x in sentences:
    for i in range(len(cross_ref)):
        if cross_ref[i].text in x:
            important_sent.append(x)

In [12]:
def remove_duplicate(alist):
    return list(set(alist))
chem_sent = remove_duplicate(important_sent)
print(chem_sent)

['(4a-4c) The highly selective adsorption of CO2 over CH4 and N2 suggests that 1 may be useful for methane purification and carbon capture.', 'To our knowledge, among numerous MOFs, only a few frameworks have shown the selectivities of >20 for CO2/CH4 and 200 for CO2/N2 at near room temperature.', '(6)Herein, we report a luminescent microporous MOF, (Me2NH2)6[In10(TTCA)12]·24DMF·15H2O (1; TTCA = triphenylene-2,6,10-tricarboxylate and DMF = N,N-dimethylformamide), that exhibits highly selective CO2 adsorption over CH4 and N2 and selective sensing of the nitro explosive 2,4,6-trinitrophenol (TNP).Solvothermal reaction of triphenylene-2,6,10-tricarboxylic acid (H3TTCA)(7) with In(NO3)3·H2O and hydrochloric acid in DMF and 1,4-dioxane at 170 °C for 72 h led to the formation of colorless block-shaped crystals of 1.', 'Using the ideal absorbed solution theory,(11) the selectivity for a 50:50 CO2/CH4 mixture at 273 K and 1 atm is 24 (17 at 298 K), and that for a 15:85 CO2/N2 mixture at 273 K 

In [13]:
words = ['adsorption', 'sorption', 'adsorb', 'adsorbent', 'adsorbate', 'loading']
iso_sent = []
for x in chem_sent:
    for word in words:
        if word in x:
            iso_sent.append(x)
iso_sent = remove_duplicate(iso_sent)
print(iso_sent)

['The selective sorption of CO2 rather than CH4 and N2 can be attributed to the quadrupole moment of CO2 (−1.4 × 10–39 cm2), which generates specific interactions with the framework.', 'Interestingly, 1 shows highly selective gas adsorption for CO2 over CH4 and N2 at 273 and 298 K. The CO2 uptakes at 1 atm reach 105.2 cm3 g–1 (4.69 mmol g–1, 20.6 wt %) at 273 K and 69.0 cm3 g–1 (3.08 mmol g–1, 13.5 wt %) at 298 K. For comparison, the CH4 and N2 uptakes are 30.2/18.3 and 7.0/4.3 cm3 g–1, respectively, under the same conditions (Figures 3a and S5 in the SI).', '(4a-4c) The highly selective adsorption of CO2 over CH4 and N2 suggests that 1 may be useful for methane purification and carbon capture.', 'N2 and H2 adsorption isotherms of 1 at 77 K and CO2 adsorption isotherm of 1 at 195 K.To further examine the gas storage capacity of 1, we also measured the CO2, CH4, and N2 sorptions at near room temperature.', 'As shown in Figure 2, N2 adsorption measurement for 1 at 77 K and 1 atm revealed

In [29]:
stop_words = set(stopwords.words('english'))


word_tokens = word_tokenize(iso_sent[1])

filtered_sentence = [w for w in word_tokens if not w in stop_words]

filtered_sentence = []

for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)
        
print(word_tokens)
print(filtered_sentence)

['Interestingly', ',', '1', 'shows', 'highly', 'selective', 'gas', 'adsorption', 'for', 'CO2', 'over', 'CH4', 'and', 'N2', 'at', '273', 'and', '298', 'K.', 'The', 'CO2', 'uptakes', 'at', '1', 'atm', 'reach', '105.2', 'cm3', 'g–1', '(', '4.69', 'mmol', 'g–1', ',', '20.6', 'wt', '%', ')', 'at', '273', 'K', 'and', '69.0', 'cm3', 'g–1', '(', '3.08', 'mmol', 'g–1', ',', '13.5', 'wt', '%', ')', 'at', '298', 'K.', 'For', 'comparison', ',', 'the', 'CH4', 'and', 'N2', 'uptakes', 'are', '30.2/18.3', 'and', '7.0/4.3', 'cm3', 'g–1', ',', 'respectively', ',', 'under', 'the', 'same', 'conditions', '(', 'Figures', '3a', 'and', 'S5', 'in', 'the', 'SI', ')', '.']
['Interestingly', ',', '1', 'shows', 'highly', 'selective', 'gas', 'adsorption', 'CO2', 'CH4', 'N2', '273', '298', 'K.', 'The', 'CO2', 'uptakes', '1', 'atm', 'reach', '105.2', 'cm3', 'g–1', '(', '4.69', 'mmol', 'g–1', ',', '20.6', 'wt', '%', ')', '273', 'K', '69.0', 'cm3', 'g–1', '(', '3.08', 'mmol', 'g–1', ',', '13.5', 'wt', '%', ')', '298', 

In [27]:
pos3 = nltk.pos_tag(filtered_sentence)
print(pos3)

[('Interestingly', 'RB'), (',', ','), ('1', 'CD'), ('shows', 'NNS'), ('highly', 'RB'), ('selective', 'JJ'), ('gas', 'NN'), ('adsorption', 'NN'), ('CO2', 'NNP'), ('CH4', 'NNP'), ('N2', 'NNP'), ('273', 'CD'), ('298', 'CD'), ('K.', 'NNP'), ('The', 'DT'), ('CO2', 'NNP'), ('uptakes', 'VBZ'), ('1', 'CD'), ('atm', 'NN'), ('reach', 'NN'), ('105.2', 'CD'), ('cm3', 'NN'), ('g–1', 'NN'), ('(', '('), ('4.69', 'CD'), ('mmol', 'NN'), ('g–1', 'NN'), (',', ','), ('20.6', 'CD'), ('wt', 'NN'), ('%', 'NN'), (')', ')'), ('273', 'CD'), ('K', 'NNP'), ('69.0', 'CD'), ('cm3', 'NN'), ('g–1', 'NN'), ('(', '('), ('3.08', 'CD'), ('mmol', 'NN'), ('g–1', 'NN'), (',', ','), ('13.5', 'CD'), ('wt', 'NN'), ('%', 'NN'), (')', ')'), ('298', 'CD'), ('K.', 'NNP'), ('For', 'IN'), ('comparison', 'NN'), (',', ','), ('CH4', 'NNP'), ('N2', 'NNP'), ('uptakes', 'VBZ'), ('30.2/18.3', 'CD'), ('7.0/4.3', 'CD'), ('cm3', 'NN'), ('g–1', 'NN'), (',', ','), ('respectively', 'RB'), (',', ','), ('conditions', 'NNS'), ('(', '('), ('Figures'

In [14]:
keyword_figs = ['figure', 'fig', 'fig.', 'figures']
fig = []
for x in iso_sent:
    for y in keyword_figs:
        start_index = x.lower().find(y)
        int(start_index)
        end_index = start_index + (len(y) + 3)
        fig.append(x[start_index:end_index])

In [15]:
correct_fig = []
for z in fig:
    if len(z) > 3:
        correct_fig.append(z)
def hasNumbers(inputString):
    return any(char.isdigit() for char in inputString)
for z in correct_fig:
    if hasNumbers(z) == False:
        correct_fig.remove(z)
print(correct_fig)

['Figures 3', 'Figures 3a', 'Figure 2,', 'Figure 2.', 'Figure S9', 'Figures S', 'Figures S7']


In [16]:
from chemdataextractor.nlp.tokenize import ChemWordTokenizer
cwt = ChemWordTokenizer()
sent = Sentence(iso_sent[0])
s = sent.pos_tagged_tokens
print(s)
from chemdataextractor.nlp.pos import ChemCrfPosTagger
cpt = ChemCrfPosTagger()


[('The', 'DT'), ('selective', 'JJ'), ('sorption', 'NN'), ('of', 'IN'), ('CO2', 'NN'), ('rather', 'RB'), ('than', 'IN'), ('CH4', 'NN'), ('and', 'CC'), ('N2', 'NN'), ('can', 'MD'), ('be', 'VB'), ('attributed', 'VBN'), ('to', 'TO'), ('the', 'DT'), ('quadrupole', 'JJ'), ('moment', 'NN'), ('of', 'IN'), ('CO2', 'NN'), ('(', '-LRB-'), ('−', 'HYPH'), ('1.4', 'CD'), ('×', 'SYM'), ('10', 'CD'), ('–', 'HYPH'), ('39', 'CD'), ('cm2', 'NN'), (')', '-RRB-'), (',', ','), ('which', 'WDT'), ('generates', 'VBZ'), ('specific', 'JJ'), ('interactions', 'NNS'), ('with', 'IN'), ('the', 'DT'), ('framework', 'NN'), ('.', '.')]


In [17]:
import inspect
pos = []
pos = nltk.pos_tag(iso_sent[0].split())
    
print(pos)

[('The', 'DT'), ('selective', 'JJ'), ('sorption', 'NN'), ('of', 'IN'), ('CO2', 'NNP'), ('rather', 'RB'), ('than', 'IN'), ('CH4', 'NNP'), ('and', 'CC'), ('N2', 'NNP'), ('can', 'MD'), ('be', 'VB'), ('attributed', 'VBN'), ('to', 'TO'), ('the', 'DT'), ('quadrupole', 'JJ'), ('moment', 'NN'), ('of', 'IN'), ('CO2', 'NNP'), ('(−1.4', 'NNP'), ('×', 'VBD'), ('10–39', 'CD'), ('cm2),', 'NN'), ('which', 'WDT'), ('generates', 'VBZ'), ('specific', 'JJ'), ('interactions', 'NNS'), ('with', 'IN'), ('the', 'DT'), ('framework.', 'NN')]


In [18]:

pos2 = []
pos2 = nltk.pos_tag(iso_sent[3].split())
    
print(pos2)

[('N2', 'NNP'), ('and', 'CC'), ('H2', 'NNP'), ('adsorption', 'NN'), ('isotherms', 'NNS'), ('of', 'IN'), ('1', 'CD'), ('at', 'IN'), ('77', 'CD'), ('K', 'NNP'), ('and', 'CC'), ('CO2', 'NNP'), ('adsorption', 'NN'), ('isotherm', 'NN'), ('of', 'IN'), ('1', 'CD'), ('at', 'IN'), ('195', 'CD'), ('K.To', 'NNP'), ('further', 'RB'), ('examine', 'VBP'), ('the', 'DT'), ('gas', 'NN'), ('storage', 'NN'), ('capacity', 'NN'), ('of', 'IN'), ('1,', 'CD'), ('we', 'PRP'), ('also', 'RB'), ('measured', 'VBD'), ('the', 'DT'), ('CO2,', 'NNP'), ('CH4,', 'NNP'), ('and', 'CC'), ('N2', 'NNP'), ('sorptions', 'NNS'), ('at', 'IN'), ('near', 'IN'), ('room', 'NN'), ('temperature.', 'NN')]


In [27]:
nltk.help.upenn_tagset()

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or